In [0]:


import psycopg2
import datetime
import pandas as pd

citus_dsn_ = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db="aa_citus_db",
        user="citus_bdp_usage_qa",
        host="10.2.10.132",
        password="dNzWtSV3pKTx",
        port=5432
    )
)

def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list

begin_date = datetime.datetime.strptime("2019-01-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2019-01-30", '%Y-%m-%d')
DATE_LIST = get_date_list(begin_date, end_date, "D")



metrics = ["free_download"]

# print "%table \tdate\tcount"
print "%table {}\t{}".format("date", "count")

for date_str in DATE_LIST:
    date_count = {}
    for metric in metrics:
        sql = 'SELECT date, device_code, country_code, category_id, count({metric}),max({metric}) FROM store.store_app_rank_fact_v1  WHERE date = \'{date_str}\' AND ( granularity = \'hourly\' ) AND ( hour = 23 ) AND ({metric} is not null)  GROUP BY country_code, date, device_code, country_code, category_id ORDER BY date ASC'.format(
            metric=metric, date_str=date_str)

        test_result = query(citus_dsn_, sql)
        date_count[metric] = 0
        for result in test_result:
            date, device_code, country_code, category_id, count, max = result
            if count != 0 and count != max:
                date_count[metric] += 1
                # print ",".join([date.strftime("%Y-%m-%d"), device_code, country_code, str(category_id), metric, str(count),str(max)])
    total_count = sum([date_count[key] for key in date_count])
    print "\t".join([date_str, str(total_count)])

print "end"

In [0]:


df = spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.0/platform=2/process_date=2019-10-14/country=AU/language=en/part-00000-fab743d5-e8ff-4f23-86ff-92bacb3fdccd-c000.snappy.parquet")
df.printSchema()
df.createOrReplaceTempView("review")
print(">>>>>>>>>>>>>>>")
# df.select('rating').rdd.map(lambda x:(x[0],1)).toDF().show()

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.types import ArrayType
def _append(x):
    return [float(val)**2 for val in x]
append_udf = udf(lambda z: _append(z), ArrayType(IntegerType()))

def square_list(x):
    x.append(0)
    return [a for a in x]

square_list_udf = udf(lambda y: square_list(y), ArrayType(IntegerType()))


df.select('topic_ids', square_list_udf('topic_ids')).show()


# df.withColumn('review_vaa',append_udf('topic_ids')).show()



# from pyspark.sql import functions as F

# spark.sql("select * from review where rating =1").show(10)






In [0]:


print "%table {}\t{}".format("date", "count")

for x in range(10):
    print str(x) ,"\t", str(2*x)
print "end"    

In [0]:
%%sh
ls -al


whoami

In [0]:


wordscounts = [('cat',2),('cat',2),('elephant',1),('rat',2)]
wordCountRDD = sc.parallelize(wordscounts)
print (wordCountRDD.collect())

totalCount = wordCountRDD.map(lambda (x,y):y).reduce(lambda x,y:x+y)

print(totalCount)


print(wordCountRDD.groupByKey().map(lambda (k,v):(k,sum(v))).collect())



print(wordCountRDD.groupByKey().map(lambda (k,v):(k,sum(v))).map(lambda (x,y):y).reduce(lambda x,y:x+y))

In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-qa/tom/gameiq/2020-03-13/


In [0]:
%%sh
